In [ ]:
spark.sql(
    """
    Drop DATABASE PL7G1_SILVER CASCADE
    """
)

In [ ]:
spark.sql(
    """
    Drop table if exists PL7G1_SILVER.instalacoes
    """
)

In [ ]:
spark.sql(
    """
   SHOW tables from PL7G1_SILVER
   """
).show()


In [ ]:
pip install delta-spark

In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import LongType, StringType, IntegerType, StructField, StructType, FloatType, DoubleType
from delta import *

In [2]:
# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

#para usarmos o delta tempos de por as ultimas linhas do config para importar bibliotecas
builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
# you can choose any location in HDFS, just be organized 
# Your data lake will grow with time and will become a swamp
spark.sql(
    """
    CREATE DATABASE PL7G1_SILVER LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db/'
    """
)

In [ ]:
spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_SILVER.instalacoes (
        county VARCHAR(50),
        municipality VARCHAR(50),
        dEC_ID VARCHAR(50),
        facility_Name VARCHAR(100),
        sIC_Code VARCHAR(50),
        vOC DOUBLE,
        nOx DOUBLE,
        cO DOUBLE,
        cO2 DOUBLE,
        particulates DOUBLE,
        pM10 DOUBLE,
        pM2_5 DOUBLE,
        hAPS DOUBLE,
        sO2 DOUBLE,
        location  VARCHAR(70)
    )
   
    
    USING DELTA
    PARTITIONED BY (
        year INT
    )
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db/instalacoes'
    """
)

In [ ]:
spark.sql(
    """
    DESCRIBE FORMATTED PL7G1_SILVER.instalacoes
    """
).toPandas()

In [ ]:
spark.sql(
    """
    SELECT *
    FROM PL7G1_SILVER.instalacoes
    """
).show()

In [115]:
hdfs_path = "hdfs://hdfs-nn:9000/Poluicao_ar/Bronze/Title_V_Emissions_Inventory__Beginning_2010.csv"

customSchema = StructType([
    StructField("year", IntegerType(), True),
    StructField("county", StringType(), True),
    StructField("municipality", StringType(), True),
    StructField("dEC_ID", StringType(), True),
    StructField("facility_Name", StringType(), True),
    StructField("sIC_Code", StringType(), True),
    StructField("vOC", DoubleType(), True),
    StructField("nOx", DoubleType(), True),
    StructField("cO", DoubleType(), True),
    StructField("cO2", DoubleType(), True),
    StructField("particulates", DoubleType(), True),
    StructField("pM10", DoubleType(), True),
    StructField("pM2_5", DoubleType(), True),
    StructField("hAPS", DoubleType(), True),
    StructField("sO2", DoubleType(), True),
    StructField("location", StringType(), True)
]) 

instalacoes = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)
instalacoes.show()
instalacoes.printSchema()

+----+----------+--------------------+----------+--------------------+--------+-----+------+------+----------+------------+-----+-----+----+-----+--------------------+
|year|    county|        municipality|    dEC_ID|       facility_Name|sIC_Code|  vOC|   nOx|    cO|       cO2|particulates| pM10|pM2_5|hAPS|  sO2|            location|
+----+----------+--------------------+----------+--------------------+--------+-----+------+------+----------+------------+-----+-----+----+-----+--------------------+
|2012|   SUFFOLK|               ISLIP|1472801386|  CEDAR GRAPHICS IGI|    2752| 8.77|  0.06|  0.05|     76.38|         0.0|  0.0|  0.0| 0.0|  0.0|POINT (-73.210578...|
|2017|     BRONX|               BRONX|2600100028|       TRACEY TOWERS|    6513|  0.0|   0.0|   0.0|       0.0|         0.0|  0.0|  0.0| 0.0|  0.0|POINT (-73.920265...|
|2017|     BRONX|               BRONX|2600500115|BRONX PSYCHIATRIC...|    8063|  0.0|   0.0|   0.0|       0.0|         0.0|  0.0|  0.0| 0.0|  0.0|POINT (-73.920

In [116]:
#código para substituir valores nulos por "sem informação"
from pyspark.sql.functions import when, col, concat, lit

instalacoes = instalacoes.withColumn(
    "Location",
    when((col("Location").isNull()),"Sem informação").
    otherwise(col("Location")))


In [117]:
#selecionar sso os conty's necessários
instalacoes = instalacoes.where((col('county') == 'QUEENS') | (col('county') == 'KINGS') | (col('county') == 'BRONX') | (col('county') == 'NEW YORK') | (col('county') == 'RICHMOND'))


In [118]:
#escrever o nome dos countys de outra maneira para estarmos todos de acordo entre datasets
from pyspark.sql.functions import when, col, concat, lit

instalacoes = instalacoes.withColumn(
    "county",
    when((col("county") == "KINGS"), "Brooklyn").  
    when((col("county") == "RICHMOND"), "Staten Island").
    when((col("county") == "NEW YORK"), "Manhattan").
    when((col("county") == "QUEENS"), "Queens").
    otherwise(col("county")))



In [119]:
#escrever com letras maiusculas
from pyspark.sql.functions import lower, col, initcap

instalacoes=instalacoes.select(initcap(col('municipality')).alias('municipality'),('county'),('year'),('dEC_ID'),('facility_Name'), ('sIC_Code'),('vOC'),('nOx'),('cO'),('cO2'),('particulates'),('pM10'),('pM2_5'),('hAPS'),('sO2'),('Location'))

instalacoes=instalacoes.select(initcap(col('county')).alias('county'),('year'),('municipality'),('dEC_ID'),('facility_Name'), ('sIC_Code'),('vOC'),('nOx'),('cO'),('cO2'),('particulates'),('pM10'),('pM2_5'),('hAPS'),('sO2'),('Location'))


In [120]:
#reordenar colunas
instalacoes = instalacoes[[('year'),('county'),('municipality'),('dEC_ID'),('facility_Name'), ('sIC_Code'),('vOC'),('nOx'),('cO'),('cO2'),('particulates'),('pM10'),('pM2_5'),('hAPS'),('sO2'),('Location')]] # setting column's order

In [121]:
#contar as linhas de county's 
instalacoes.select('county').count()

955

In [122]:
#renomear colunas
instalacoes = instalacoes.selectExpr("year as Year", "county as Borough", "municipality as Municipality", "dEC_ID as DEC_ID", "facility_Name as Facility_Name", "sIC_Code as SIC_Code",
                                        "vOC as VOC", "nOx as NOx", "cO as CO", "cO2 as CO2", "particulates as Particulates", "pM10 as PM10", "pM2_5 as PM2_5", "hAPS as HAPS", 
                                        "sO2 as SO2", "Location as Location")

In [123]:
instalacoes.toPandas()

Year        Borough          Municipality      DEC_ID  \
0    2017          Bronx                 Bronx  2600100028   
1    2017          Bronx                 Bronx  2600500115   
2    2012       Brooklyn       Brooklyn (6104)  2610400015   
3    2018       Brooklyn              Brooklyn  2610700012   
4    2011  Staten Island  Staten Island (6403)  2640300014   
..    ...            ...                   ...         ...   
950  2014          Bronx          Bronx (6005)  2600500125   
951  2016       Brooklyn              Brooklyn  2610100185   
952  2014       Brooklyn       Brooklyn (6104)  2610401298   
953  2012         Queens         Queens (6304)  2630401377   
954  2018       Brooklyn              Brooklyn  2610300760   

                             Facility_Name SIC_Code    VOC     NOx      CO  \
0                            TRACEY TOWERS     6513   0.00    0.00    0.00   
1                 BRONX PSYCHIATRIC CENTER     8063   0.00    0.00    0.00   
2               NYC-HH - WOODHULL HOSPITAL     8062   0.65    9.88    7.05   
3           NYC-HH - CONEY ISLAND HOSPITAL     8062   0.15    0.26    0.44   
4           ARTHUR KILL GENERATING STATION     4911  48.03  772.09  733.62   
..                                     ...      ...    ...     ...     ...   
950                              BRONX ZOO     8422   0.23   36.07    5.02   
951  BROOKLYN NAVY YARD COGENERATION PLANT     4911   0.03   41.69    6.78   
952         WYCKOFF HEIGHTS MEDICAL CENTER     8062   0.93    3.69    2.91   
953                      VERNON BLVD PLANT     4911   3.02    4.06    2.36   
954              MAIMONIDES MEDICAL CENTER     8062   0.56   13.39   10.94   

            CO2  Particulates   PM10  PM2_5  HAPS   SO2  \
0          0.00          0.00   0.00   0.00   0.0  0.00   
1          0.00          0.00   0.00   0.00   0.0  0.00   
2       9513.34          0.68   0.68   0.60   0.0  0.42   
3        691.07          0.05   0.03   0.02   0.0  0.00   
4    1047706.24         66.38  66.06  66.00   0.0  5.29   
..          ...           ...    ...    ...   ...   ...   
950    23841.84          0.49   0.15   0.34   0.0  0.09   
951  1174765.47          0.06   0.06   0.00   0.0  5.97   
952     3670.26          0.23   0.00   0.23   0.0  0.33   
953    42083.19          2.61   2.61   2.61   0.0  0.25   
954    15507.61          0.01   0.00   0.00   0.0  0.11   

                         Location  
0    POINT (-73.920265 40.826153)  
1    POINT (-73.920265 40.826153)  
2    POINT (-73.990996 40.692529)  
3    POINT (-73.990996 40.692529)  
4    POINT (-74.076203 40.643498)  
..                            ...  
950  POINT (-73.920265 40.826153)  
951  POINT (-73.990996 40.692529)  
952  POINT (-73.990996 40.692529)  
953  POINT (-73.817302 40.759776)  
954  POINT (-73.990996 40.692529)  

[955 rows x 16 columns]

In [124]:
#separar o a coluna das coordenadas para latitude e longitude
from pyspark.sql.functions import when, col, concat, lit, split

instalacoes= instalacoes.withColumn("longitude",split(col("Location")," ").getItem(1)) \
.withColumn("latitude",split(col("Location")," ").getItem(2)) \
.drop("Location")

instalacoes.printSchema()
instalacoes.show(30)

root
 |-- Year: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Municipality: string (nullable = true)
 |-- DEC_ID: string (nullable = true)
 |-- Facility_Name: string (nullable = true)
 |-- SIC_Code: string (nullable = true)
 |-- VOC: double (nullable = true)
 |-- NOx: double (nullable = true)
 |-- CO: double (nullable = true)
 |-- CO2: double (nullable = true)
 |-- Particulates: double (nullable = true)
 |-- PM10: double (nullable = true)
 |-- PM2_5: double (nullable = true)
 |-- HAPS: double (nullable = true)
 |-- SO2: double (nullable = true)
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)



In [126]:
##tirar o ) que tinha ficado na string
from pyspark.sql import functions as F
instalacoes = instalacoes.withColumn("latitude", F.translate("latitude", ")", " "))
instalacoes.show()

+----+-------------+--------------------+----------+--------------------+--------+-----+------+------+----------+------------+-----+-----+----+-----+-----------+----------+
|Year|      Borough|        Municipality|    DEC_ID|       Facility_Name|SIC_Code|  VOC|   NOx|    CO|       CO2|Particulates| PM10|PM2_5|HAPS|  SO2|  longitude|  latitude|
+----+-------------+--------------------+----------+--------------------+--------+-----+------+------+----------+------------+-----+-----+----+-----+-----------+----------+
|2017|        Bronx|               Bronx|2600100028|       TRACEY TOWERS|    6513|  0.0|   0.0|   0.0|       0.0|         0.0|  0.0|  0.0| 0.0|  0.0|(-73.920265|40.826153 |
|2017|        Bronx|               Bronx|2600500115|BRONX PSYCHIATRIC...|    8063|  0.0|   0.0|   0.0|       0.0|         0.0|  0.0|  0.0| 0.0|  0.0|(-73.920265|40.826153 |
|2012|     Brooklyn|     Brooklyn (6104)|2610400015|NYC-HH - WOODHULL...|    8062| 0.65|  9.88|  7.05|   9513.34|        0.68| 0.68|  0

In [128]:
#tirar o ( que tinha ficado na string
from pyspark.sql import functions as F
instalacoes = instalacoes.withColumn("longitude", F.translate("longitude", "(", " "))
instalacoes.show()

+----+-------------+--------------------+----------+--------------------+--------+-----+------+------+----------+------------+-----+-----+----+-----+-----------+----------+
|Year|      Borough|        Municipality|    DEC_ID|       Facility_Name|SIC_Code|  VOC|   NOx|    CO|       CO2|Particulates| PM10|PM2_5|HAPS|  SO2|  longitude|  latitude|
+----+-------------+--------------------+----------+--------------------+--------+-----+------+------+----------+------------+-----+-----+----+-----+-----------+----------+
|2017|        Bronx|               Bronx|2600100028|       TRACEY TOWERS|    6513|  0.0|   0.0|   0.0|       0.0|         0.0|  0.0|  0.0| 0.0|  0.0| -73.920265|40.826153 |
|2017|        Bronx|               Bronx|2600500115|BRONX PSYCHIATRIC...|    8063|  0.0|   0.0|   0.0|       0.0|         0.0|  0.0|  0.0| 0.0|  0.0| -73.920265|40.826153 |
|2012|     Brooklyn|     Brooklyn (6104)|2610400015|NYC-HH - WOODHULL...|    8062| 0.65|  9.88|  7.05|   9513.34|        0.68| 0.68|  0

In [131]:
#dar save
instalacoes \
    .select("Year","Borough","Municipality","DEC_ID","Facility_Name","SIC_Code","VOC","NOx","CO","CO2", "Particulates", "PM10", "PM2_5", "HAPS", "SO2", "longitude", "latitude") \
    .write \
    .mode("overwrite") \
    .partitionBy("year") \
    .option("overwriteSchema", "true")\
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db/instalacoes")

In [132]:
spark.sql(
    """
    SELECT * FROM PL7G1_SILVER.instalacoes
    """
).show()

+----+-------------+--------------------+----------+--------------------+--------+------+------+------+----------+------------+-----+-----+----+------+-----------+----------+
|Year|      Borough|        Municipality|    DEC_ID|       Facility_Name|SIC_Code|   VOC|   NOx|    CO|       CO2|Particulates| PM10|PM2_5|HAPS|   SO2|  longitude|  latitude|
+----+-------------+--------------------+----------+--------------------+--------+------+------+------+----------+------------+-----+-----+----+------+-----------+----------+
|2011|Staten Island|Staten Island (6403)|2640300014|ARTHUR KILL GENER...|    4911| 48.03|772.09|733.62|1047706.24|       66.38|66.06| 66.0| 0.0|  5.29| -74.076203|40.643498 |
|2011|    Manhattan|    Manhattan (6205)|2620500182|MUTUAL REDEVELOPM...|    6513| 17.04| 56.42|  51.3|  15685.94|       10.38|10.37|10.37| 0.0|  6.06| -74.003804|40.753259 |
|2011|    Manhattan|    Manhattan (6206)|2620600096|EAST RIVER HOUSIN...|    6513|  0.82| 45.26|  5.14|   26535.3|        5.1

In [ ]:
spark.stop()